In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time


In [ ]:
URL = "https://www.amazon.com/s?k=amharic+books&i=stripbooks-intl-ship&crid=13KU6MP8CV12F&sprefix=amharic+book%2Cstripbooks-intl-ship%2C337&ref=nb_sb_noss_2"

In [ ]:
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Linux; Android 13; Pixel 6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Mobile Safari/537.36',
    'Accept-Language': 'am, en-US, en;q=0.5'
}


In [ ]:
# Get the search page
webpage = requests.get(URL, headers=HEADERS)
webpage
# soup = BeautifulSoup(webpage.content, "html.parser")

In [ ]:
# Fetch links as List of Tag Objects
links = soup.find_all("a", attrs={'class':'a-link-normal s-faceout-link a-text-normal'})

In [ ]:
# Extract product links
product_list = []
for link in links:
    href = link.get('href')
    if href:
        product_list.append("https://amazon.com" + href)

In [ ]:
# Initialize lists to store data
titles, authors, ratings, prices = [], [], [], []

In [ ]:
# Loop through each product link and extract details
for link in product_list:
    try:
        response = requests.get(link, headers=HEADERS)
        time.sleep(2)  # Delay to prevent getting blocked
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Extract title
            try:
                title = soup.find("span", attrs={"id": "productTitle"}).text.strip()
            except AttributeError:
                title = "N/A"
            
            # Extract author
            try:
                author = soup.find("a", attrs={"class": "a-link-normal contributorNameID"}).text.strip()
            except AttributeError:
                author = "N/A"
            
            # Extract rating
            try:
                rating = soup.find("span", attrs={"class": "a-icon-alt"}).text.strip()
            except AttributeError:
                rating = "No rating"
            
            # Extract price
            try:
                price = soup.find("span", attrs={"class": "a-size-medium a-color-price"}).text.strip()
            except AttributeError:
                price = "Price not available"
            
            # Append the extracted data to the lists
            titles.append(title)
            authors.append(author)
            ratings.append(rating)
            prices.append(price)

        else:
            print(f"Failed to retrieve {link} - Status code: {response.status_code}")

    except requests.RequestException as e:
        print(f"An error occurred with {link}: {e}")


In [ ]:
# Create a DataFrame with the scraped data
df = pd.DataFrame({
    'Title': titles,
    'Author': authors,
    'Rating': ratings,
    'Price': prices
})

# Save the data to a CSV file
df.to_csv('amazon_books.csv', index=False)

print(df)